## Docker Buildx

1. **Unterliegende Technologie**  
   „`docker build`“ nutzt standardmässig den klassischen Moby-/Docker‑Daemon‑Builder, während „`docker buildx`“ immer BuildKit als Engine verwendet. Buildx ist im Kern ein CLI‑Plugin, das BuildKit‑Funktionen freischaltet, während „`docker build`“ nur optional auf BuildKit zugreifen kann, wenn es explizit aktiviert wurde.

2. **Multi‑Plattform‑Builds**  
   Mit Buildx lassen sich Images für mehrere Architekturen in einem Aufruf erstellen (z. B. `--platform linux/amd64,linux/arm64`), während das klassische „`docker build`“ nur für die lokale Architektur baut und keine integrierte Mehrfach‑Plattform‑Unterstützung bietet.

3. **Erweiterte Caching‑Möglichkeiten**  
   Buildx unterstützt feingranulare Cache‑Kontrolle über `--cache-to` und `--cache-from` mit verschiedenen Backends (Registry, S3, lokal, GitHub Actions etc.), um Caches zwischen Builds und Umgebungen zu teilen. Das herkömmliche „`docker build`“ verwendet nur den internen lokalen Layer‑Cache des Daemons und bietet keine Remote‑Cache‑Exports.

4. **Builder‑Instanzen und Treiber**  
   Buildx erlaubt das Anlegen mehrerer, entkoppelter Builder‑Instanzen (z. B. mit dem Docker‑Container‑ oder Kubernetes‑Treiber), lokal oder remote. So kann man verschiedene Umgebungen parallel verwalten. „`docker build`“ dagegen ist immer an den einen, lokalen Docker‑Daemon gebunden.

5. **Flexible Ausgabe‑ und Exportformen**  
   Mit Buildx lassen sich Build‑Ergebnisse nicht nur ins lokale Image‑Store laden, sondern direkt als Tar‑Archiv, OCI‑Layout oder in ein Registry‑Image exportieren (`--output`‑Flag), und sogar mehrere Exporter parallel nutzen. Das klassische „`docker build`“ exportiert standardmäßig nur in den lokalen Daemon und kennt keine `--output`‑Vielfalt.

6. **Bake‑Orchestrierung**  
   Buildx bietet über den Befehl `docker buildx bake` eine deklarative, HCL‑basierte Möglichkeit, komplexe Multi‑Target‑Builds parallel und konsistent zu definieren. Für einfache Einzel‑Dockerfiles gibt es kaum Unterschiede zu CLI‑Flags, aber Bake skaliert viel besser für grosse Projekte und Team‑Setups.

- - - 
### Auto Shop 

Als Beispielprojekt verwenden wir Auto Shop GmbH

In [ ]:
%%bash
git clone https://gitlab.com/ch-mc-b/autoshop-ms/app/shop.git shop-buildx

- - -

### Kind

Kind-Cluster laufen in Docker‑Containern, nutzen aber nicht automatisch den lokalen Docker‑Daemon. Du baust das Image mit Buildx ins lokale Docker‑Image‑Store und lädst es dann in Kind:

In [ ]:
%%bash
cd shop-buildx/webshop

# 1. Build in lokalen Docker‑Store (docker‑Exporter, --load)
docker buildx build -t webshop:buildx --platform linux/amd64 --load .

# 2. Import in Kind
kind load docker-image webshop:buildx --name kind

In [ ]:
%%bash
docker image ls

Anzeige der Images in Kubernetes

In [ ]:
%%bash
docker exec $(docker ps -q -f name=kind-control-plane) crictl images

- - -
## Microk8s

Du kannst das gebaute Image auch exportieren und mit Containerd importieren:

In [ ]:
%%bash

# 1. Build in lokalen Docker‑Store (docker‑Exporter, --load)
docker buildx build -t webshop:buildx --platform linux/amd64 --load .

docker save webshop:buildx | microk8s ctr image import -


Anzeigen der Images in Kubernetes

In [ ]:
%%bash
microk8s ctr image ls